In [1]:
import pandas as pd
import sqlite3
import re

In [2]:
# Start of cleaning the review dataset
conn = sqlite3.connect("yelp_data.db")
review_df = pd.read_sql(
    "SELECT * FROM review", conn)
conn.close()

review_df.tail()

,review_id,user_id,business_id,stars,text,date,review_score
4988729,7NgXAuTFiJHYbuepOPwU0w,x1QLCwZGFAjxRRw4EHc3-g,1_BVWDzi5cVqWxNe9bOMMQ,5.0,Don't misinterpret my 5-star review....I don't...,2016-04-30 01:02:34,0.8
4988730,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5.0,It is very rare for a restaurant to be this go...,2022-01-17 22:36:01,0.8
4988731,YVX1Wsa4LYxjvFwuHBb_gA,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,4.0,"Good, maybe very good. I went for lunch, so t...",2015-01-23 23:52:03,2.2
4988732,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,0.6
4988733,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47,8.0
